<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Class_04_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

##### **Module 4: ChatGPT and Large Language Models**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)

### Module 4 Material

* Part 4.1: Introduction to Large Language Models (LLMs)
* Part 4.2: Chatbots
* **Part 4.3: Image Generation with StableDiffusion**
* Part 4.4: Agentic AI


## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to /content/drive and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [ ]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    Colab = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    Colab = False

You should see the following output except your GMAIL address should appear on the last line.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_1_image01B.png)

If your GMAIL address does not appear your lesson will **not** be graded.


## **Accelerated Run-time Check**

You MUST run the following code cell to get credit for this class lesson. The code in this cell checks what hardware acceleration you are using. To run this lesson, you must be running a Graphics Processing Unit (GPU). You don't need a "high-end" GPU like the `A100` but you can select something less powerful like the `L4 GPU` to save your Colab credits.

In [ ]:
# You must run this cell second

import torch

# Check for GPU
def check_colab_gpu():
    print("=== Colab GPU Check ===")

    # Check PyTorch
    pt_gpu = torch.cuda.is_available()
    print(f"PyTorch GPU available: {pt_gpu}")

    if pt_gpu:
        print(f"PyTorch device count: {torch.cuda.device_count()}")
        print(f"PyTorch current device: {torch.cuda.current_device()}")
        print(f"PyTorch device name: {torch.cuda.get_device_name()}")
        print("You are good to go!")

    else:
        print("No compatible device found")
        print("WARNING: You must run this assigment using either a GPU to earn credit")
        print("Change your RUNTIME now and start over!")

check_colab_gpu()

If you current `Runtime` is correct you should see something _similar_ to the following output

```text
=== Colab GPU Check ===
PyTorch GPU available: True
PyTorch device count: 1
PyTorch current device: 0
PyTorch device name: NVIDIA L4
You are good to go!
```
However, if you received a warning message, you must go back and change your `Runtime` now before you continue.

### **YouTube Introduction to Stable Diffusion**

Run the next cell to see short introduction to Stable Diffusion. This is a suggested, but optional, part of the lesson.

In [ ]:
from IPython.display import HTML
video_id =  "QdRP9pO89MY"

HTML(f"""
<iframe width="560" height="315"
  src="https://www.youtube.com/embed/{video_id}"
  title="YouTube video player"
  frameborder="0"
  allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture"
  allowfullscreen
  referrerpolicy="strict-origin-when-cross-origin"> </iframe>
""")

# **Stable Diffusion**

**Stable Diffusion** is a deep learning model for **generative image synthesis**. It belongs to the class of **latent diffusion models (LDMs)**, which generate high-quality images from text prompts by operating in a compressed latent space rather than pixel space. This approach significantly reduces computational cost while maintaining image fidelity.

Stable Diffusion is trained on large-scale image-text datasets and uses a combination of:

- **Variational Autoencoders (VAEs)**: To encode images into a latent space.
- **U-Net architecture**: For denoising latent representations.
- **Text encoders (e.g., CLIP or BERT)**: To condition image generation on natural language prompts.

The model works by iteratively denoising a random latent vector, guided by a text prompt, until a coherent image emerges.

### **Key Features**

- **Text-to-image synthesis**: Generate images from descriptive text.
- **Image-to-image translation**: Modify existing images using prompts.
- **Inpainting and outpainting**: Fill in missing regions or expand images.
- **Custom fine-tuning**: Adapt the model to domain-specific data.

### **Applications in Computational Biology**

Stable Diffusion can be a powerful tool for computational biologists in several ways:

#### **1. Scientific Visualization**
Generate illustrative figures for:
- Molecular structures
- Cellular processes
- Pathways and interactions
- Anatomical diagrams

This can enhance presentations, publications, and educational materials.

#### **2. Data Augmentation**
Use synthetic biological images to:
- Augment training datasets for machine learning models
- Improve robustness in image classification tasks (e.g., histopathology, microscopy)

#### **3. Hypothesis Communication**
Translate complex biological hypotheses into visual representations for:
- Grant proposals
- Interdisciplinary collaboration
- Public outreach

#### **4. Custom Model Training**
Fine-tune Stable Diffusion on domain-specific datasets (e.g., microscopy images, protein structures) to:
- Generate realistic biological imagery
- Explore latent space representations of biological phenomena

#### **5. Interactive Exploration**
Use prompt-based generation to explore:
- Morphological variations
- Evolutionary traits
- Synthetic biology designs

## **Text to Images with StableDiffusion**

We will now see how to use Stable Diffusion to create various images from textual prompts. There will be four settings that we will deal with as we generate these images.

* **model**: We will use the trained/finetuned model. Different models are optimized for different types of images.
* **prompt**: Text that you provide to describe what sort of image you would like created.
* **negative prompt**: Text that you describe elements that should not be present in your image.
* **seed**: The same image for the prompt/negative prompt will always be produced for the same seed. To get a different image for the same prompts, change the seed.


### **Importance of "Setting the Seed"**

In image generation models like **Stable Diffusion**, a **random seed** determines the initial noise used to create an image. This noise is gradually transformed into a coherent image based on the prompt you provide.

A **random seed** is a number used to initialize a random number generator. In Stable Diffusion:

- The model begins with a field of random noise.
- It uses your prompt to guide the transformation of this noise into an image.
- The seed controls the exact pattern of that starting noise.

Even with the **same prompt**, changing the seed changes the initial noise pattern, which leads to a **different final image**.

**Analogy**:  
Imagine sculpting clay using the same instructions. If each lump of clay starts with a different shape (seed), the final sculptures will be similar but **not identical**.

#### **Use Cases for Seeds**

- **Reproducibility**: Using the same seed and prompt will always generate the same image.
- **Exploration**: Trying different seeds lets you explore variations of the same concept.
- **Fine-tuning**: You can find a seed that gives you the best result and reuse it.



### Setup Basic Pipeline

To make use of Stable Diffusion we will use the `HuggingFace DiffusionPipeline`. When setting up the pipeline we specify to use the `CompVis/stable-diffusion-v1-4` model, which is a basic model created to be used with StableDiffusion.

The following code sets up this model and downloads it from `HuggingFace`.

In [ ]:
# Setup Basic Pipeline

from diffusers import StableDiffusionPipeline
import torch

MODEL_ID = "CompVis/stable-diffusion-v1-4"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32

pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=DTYPE,
)
pipe = pipe.to(DEVICE)
print(f"Pipeline loaded successfully on {DEVICE}.")


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image01F.png)

### Example 1: Generate Basic Image

We will begin by using Stable Diffusion (`stable-diffusion-v1-4`) to create a simple picture of a Monarch butterfly. As you will see, the image we generate will depend to a great extent on the value to random seed. For Example 1 we will set the seed = `100`.

In **Stable Diffusion**, the terms `prompt` and `negative prompt` refer to two different types of input that guide the image generation process:

**Prompt**

* This is the **main description** of what you want the model to generate.
* It includes **keywords, phrases, or detailed descriptions** of the desired scene, style, objects, characters, lighting, mood, etc.

Here is the `prompt` for Example 1:
```text
# Set prompt
prompt= """
a Monarch butterfly"""
```
**Negative Prompt**

* This is used to specify **what you _don’t_ want** in the image.
* It helps the model avoid unwanted elements, styles, or artifacts.

Here is the negative prompt for Example 1:
```type
# Set negative prompt
neg_prompt = """
signature, watermark, incomplete image
"""
```

**NSFW**

As you might imagine it is quite possible to generate images that are considered "not safe for work" (NSFW). The code in the cell below contains the following line of code that will protect you from inadvertently generating a pornographic and/or an extremely violent image:

```type
# Uncomment the next line to remove the safety checker
# pipe.safety_checker = lambda images, clip_input: (images, False)
```
If such an image is generated, you will see the following message.
```text
Potential NSFW content was detected in one or more images. A black image will be returned instead.
Try again with a different prompt and/or seed.
```

You may wish to disable this feature. To do this, uncomment the pipe.`safety_checker` line. Be cafeful, if you do disable this, as unsafe images may be generated containing NSFW themes, which might contain violence, nudity, or sexual themes.
`

In [ ]:
# Example 1: Generate basic image

import random

# Set the seed
seed = 100
seed = random.randint(0, 2**32) if seed == -1 else seed
print(f"The seed =", seed)

# Use seed to create the generator
generator = torch.Generator(device='cuda').manual_seed(int(seed))

# Set prompt
prompt= """
a Monarch butterfly"""

# Set negative prompt
neg_prompt = """
signature, watermark, incomplete image
"""

# Uncomment the next line to remove the safety checker
# pipe.safety_checker = lambda images, clip_input: (images, False)

# Generate image
pipe(prompt, negative_prompt=neg_prompt,
              width=512,height=512,generator=generator).images[0]

If the code is correct you should see something _similar_ to following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image09B.png)

### **Exercise 1A: Generate Basic Image**

In the cell below write the code to generate the same image of a Monarch butterfly generated in Example 1, but set the `seed` to the number `1604`.

In [ ]:
# Insert your code for Exercise 1A here


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image10B.png)

### **Exercise 1B: Generate Basic Image**

In the cell below write the code to generate the same image of a Monarch butterfly but this time set the seed to a **random number**. This is easy to do by just setting the seed value of `-1`.

In [ ]:
# Insert your code for Exercise 1B here



If the code is correct you should see something similar to the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image11B.png)

but it's unlikely you will see the same image since you were to use a random seed.

### **Exercise 1C: Generate Basic Image**

In the cell below write the code to generate an image of your choice. It can be any image that you like. Also, the value you pick for your `seed` is also up to you. Show your imagination!

In [ ]:
# Insert your code for Exercise 1C here



The output will depend upon your `prompt` and your seed value.

### Example 2: Generate Reference Image

The diffusion model that we have been using so far is adequate for similar images, but is less suitable for high resolution images. This is especially true when it comes to generating images with human faces.

The code in the next cell generates the face of a young Japanese woman. We will use this image as an example of a basic image that can be generated with the `stable-diffusion-v1-4`  model.


In [ ]:
# Example 2: Importance of model selection

import random

# Set the seed / Use -1 for random seed
seed = 100
seed = random.randint(0, 2**32) if seed == -1 else seed
print(f"The seed =", seed)

# Use seed to create generator
generator = torch.Generator(device='cuda').manual_seed(int(seed))

# Prompts to generate image
prompt= """
the face of a young Japanese woman"""

neg_prompt = """
signature, watermark, incomplete image
"""

# Uncomment the next line to remove the safety checker
#pipe.safety_checker = lambda images, clip_input: (images, False)

# Generate image
pipe(prompt, negative_prompt=neg_prompt, width=512,height=512,
              max_embeddings_multiples=3,generator=generator).images[0]

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image12B.png)

### **Exercise 2: Generate Reference Image**

In the next cell write the code that will generate a human face. You are free to specify the kind of face you want, male or female.  


In [ ]:
# Insert your code for Exercise 2 here




The output will depend upon your `prompt`.

# **Stable Diffusion Model Comparison: v1.4 vs v2.1**

In the previous section we used the `stable-diffusion-v1-4` model to generate basic images. In this section we will use the more advanced version of this model, stable-diffusion-2-1 which can generate more realistics images.

Here are the main differences between these two model generations.


**1. Architecture and Text Encoder**

| Feature | v1.4 (CompVis) | v2.1 (StabilityAI) |
|--------|----------------|--------------------|
| Text Encoder | CLIP ViT-L/14 | OpenCLIP ViT-H/14 |
| Architecture | Stable Diffusion 1.x | Updated architecture |
| Native Resolution | 512x512 | 768x768 |


**2. Training Data**
- **v1.4**:
  - Trained on **LAION-Aesthetics v2 5+**.
  - Fine-tuned for 225k steps.
  - Focused on aesthetic images.

- **v2.1**:
  - Trained on a **filtered subset of LAION-5B**.
  - Improved safety and quality filtering.
  - Better prompt alignment and reduced bias


**3. Output Style and Quality**

- **v1.4**:
  - Good for general-purpose image generation.
  - May struggle with complex compositions or high-resolution needs.

- **v2.1**:
  - Better at handling **complex prompts**, **realism**, and **fine details**.
  - Improved **anatomical accuracy**, lighting, and texture rendering.

**4. Use Cases**

| Model | Best For | Limitations |
|-------|----------|-------------|
| **v1.4** | Quick, general image generation | Lower resolution, less prompt nuance |
| **v2.1** | High-quality, detailed images; better realism | May interpret prompts differently due to OpenCLIP |


### Login into Huggingface

Run the next cell to log into your Hugginface account using your HF_API stored in your Colab Secrets. Information about obtaining a Huggingface API token has already been provided to you.

In [ ]:
# Log into Huggingface

from google.colab import userdata
from huggingface_hub import login

try:
    login(token=userdata.get('HF_Token'))
    print("Logged in to Hugging Face successfully.")
except Exception as e:
    print(f"Failed to log in to Hugging Face: {e}")
    print("Please check that your HF_Token is correctly stored in Colab Secrets.")


If your Huggingface toekn is correctly installed in your Colab's Secrets, you should see the following output:

```text
Logged in to Hugging Face successfully.
```

See your course Instructor or a TA if you can't get your Huggingface token to work.

### Setup Advanced Pipleline

The code in the cell below sets up the more advanced pipeline `stabilityai/stable-diffusion-2-1`.


In [ ]:
# Setup Advanced Pipeline

!pip install -q compel

from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    'runwayml/stable-diffusion-v1-5',
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

print("Pipeline loaded successfully on cuda.")


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image25F.png)

### Example 3: Realistic Image

We now generate an image with a much more complex prompt. The positive and negative prompts describe how to generate an image of a young woman. Stable diffusion prompts are usually comma separated lists of attributes to draw.

You will notice that some aspects of the prompt (e.g. `(FACE1:0.5)`) are enclosed in paranthesis; which designates that this attribute is more important. A number, near the end, separated by a colon specifies how important.

In [ ]:
# Example 3: Realistic Image

from compel import Compel
import random

# Set the seed
seed = 8
seed = random.randint(0, 2**32) if seed == -1 else seed
print(f"The seed =", seed)

# Create generator
generator = torch.Generator(device='cuda').manual_seed(int(seed))

# Create compel processor
compel = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)

# Prompt
prompt = """(woman age 26 standing by tree), (long blonde hair:1.2), ray traced shadows,
RAW, 8k, (eczema:0.7), (sub-surface scattering:1.55), (sweat:1.22), (freckles:0.55),
highly detailed skin, (Acne:0.7), (FACE1:0.5), (FACE2:1.2), (FACE3:0.85),
perfect eyes, no makeup, (skin spores:1.05), (skin spores:1.05),
ultra detailed face, ultra detailed skin, film grain, ray tracing, studio lighting"""

# Negative prompt
neg_prompt = """signature, watermark, airbrush, photoshop, plastic doll,
(ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2),
(un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon,
drawing, anime:1.2), text, close up, cropped, out of frame, worst quality,
low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers,
mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry,
dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured,
gross proportions, malformed limbs, missing arms, missing legs, extra arms,
extra legs, fused fingers, too many fingers, long neck, head wear, masculine,
obese, fat, out of frame"""

# Generate embeddings with compel
prompt_embeds = compel(prompt)
neg_embeds = compel(neg_prompt)

# Generate image using embeddings
image = pipe(
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=neg_embeds,
    width=512,
    height=512,
    generator=generator
).images[0]

image


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image14F.png)

### **Exercise 3: Realistic Image**

Generate an image of old Japanese woman in front of Mount Fuji. Set the seed to `6`.

Define your prompt as:
```text
# Prompt
prompt= """
(Japanese woman age 90+, standing before Mount Fuji), (gray thinning hair:1.2),
ray traced shadows, RAW, 8k, (deep wrinkles:1.5), (sub-surface scattering:1.55),
(age spots:1.2), highly detailed aged skin, (sagging skin:1.3), (FACE1:0.5),
(FACE2:1.2), (FACE3:0.85), cloudy eyes, no makeup, (skin texture:1.05),
ultra detailed face, ultra detailed skin, film grain, ray tracing, studio lighting
"""
```
and your negative prompt as:
```type
# Negative prompt
neg_prompt = """
signature, watermark, airbrush, photoshop, plastic doll,
(ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2),
(un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon,
drawing, anime:1.2), text, close up, cropped, out of frame, worst quality,
low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers,
mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry,
dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured,
gross proportions, malformed limbs, missing arms, missing legs, extra arms,
extra legs, fused fingers, too many fingers, long neck, head wear, masculine,
obese, fat, out of frame"""
```

The rest of the code is the same as Example 3.

In [ ]:
# Insert your code for Exercise 3 here




If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image15F.png)

## **Anime**

**Anime** is a form of animation that originated in **Japan** and has grown into a globally recognized medium. It includes a wide variety of genres, themes, and artistic styles, and is known for its unique storytelling and visual aesthetics.

#### **Key Characteristics**
- **Origin**: Japan
- **Mediums**: TV series, films, web series, OVAs (Original Video Animations)
- **Genres**: Action, Romance, Fantasy, Horror, Sci-Fi, Slice of Life, and more
- **Audience**: All age groups, from children to adults

#### **Notable Examples**
- *Spirited Away* (Studio Ghibli)
- *Naruto*
- *Attack on Titan*
- *One Piece*
- *Demon Slayer*

### **What Is Anime Style?**

**Anime style** refers to the distinctive visual and artistic elements commonly found in anime. It is characterized by stylized character designs, expressive features, and dynamic visuals.

#### **Visual Features**

| Feature              | Description                                                                 |
|----------------------|-----------------------------------------------------------------------------|
| **Eyes**             | Large, expressive, often detailed to convey emotion                         |
| **Facial Features**  | Small noses and mouths, exaggerated expressions                             |
| **Hair**             | Stylized, often colorful, with unique shapes and movement                   |
| **Body Proportions** | Can range from realistic to highly exaggerated depending on the genre       |
| **Line Work**        | Clean and sharp outlines                                                     |
| **Coloring**         | Flat or cel-shaded, with emphasis on contrast and mood                      |
| **Backgrounds**      | Often highly detailed, especially in fantasy or sci-fi settings             |

#### **Storytelling Elements**
- **Emotional depth** and character development
- **Symbolism** and metaphorical themes
- **Cultural references** to Japanese traditions, language, and society
- **Genre blending**, such as mixing romance with supernatural or comedy with horror

#### **Anime vs. Western Animation**

| Aspect              | Anime                                   | Western Animation                      |
|---------------------|------------------------------------------|----------------------------------------|
| **Art Style**       | Stylized, detailed, expressive           | Varies widely, often more cartoonish   |
| **Themes**          | Often complex and mature                 | Often geared toward children/family    |
| **Production**      | Typically serialized with long arcs      | Often episodic                         |
| **Cultural Influence** | Strong Japanese cultural elements     | Western cultural norms and humor       |


## **Anime Models**

If you would like to generate cartoon or Anime style images, the `waifu-diffusion model` will work nicely. The `Waifu Diffusion model` is a specialized AI image generation model designed to create anime-style artwork, particularly focusing on characters that resemble the "waifu" archetype—typically stylized, idealized female characters popular in anime and manga culture.

#### **Key Features:**
* **Anime-focused training data:** Trained on datasets like Danbooru (a large anime image repository).
* **Text-to-image generation:** You input a prompt like "a cute anime girl with blue hair in a school uniform" and it generates an image.
* **Customizable outputs:** You can adjust style, pose, background, and more using prompt engineering.
* **Checkpoint versions:** Includes models like Waifu Diffusion 1.3, 1.4, and 1.5, each improving quality and prompt responsiveness.

The code below loads the pipeline for the `Waifu Diffusion model`.

In [ ]:
# Setup Waifu Diffusion model pipeline

from diffusers import StableDiffusionPipeline
from compel import Compel
import torch
import random

# Load pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    "hakurei/waifu-diffusion",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

# Create compel processor
compel = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)

print("Waifu Diffusion pipeline loaded successfully on cuda.")


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image16F.png)

### Example 4: Create Anime Person

The code in the cell below creates an Anime image of a young girl.

In [ ]:
# Example 4: Create anime image

import random

# Random number seed, -1 for random seed
seed = 105
seed = random.randint(0, 2**32) if seed == -1 else seed
print(f"The seed =", seed)

# Create generator
generator = torch.Generator(device='cuda').manual_seed(int(seed))

# Prompt
prompt = (
    "best_quality (1girl:1.3) bride brown_hair closed_mouth frills (full_body:1.3) "
    "fox_ear hair_bow happy hood kimono long_sleeves red_bow smile solo tabi "
    "white_kimono wide_sleeves cherry_blossoms"
)

# Negative prompt
neg_prompt = (
    "lowres, bad_anatomy, error_body, error_hands, bad_hands, error_fingers, "
    "missing_fingers, error_legs, bad_legs, error_lighting, error_shadow, "
    "extra_digit, cropped, worst_quality, jpeg_artifacts, watermark, blurry"
)

# Generate embeddings
prompt_embeds = compel(prompt)
neg_embeds = compel(neg_prompt)

# Generate the image
image = pipe(
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=neg_embeds,
    width=512,
    height=512,
    generator=generator
).images[0]

image


If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image16B.png)

### **Exercise 4: Create Anime Person**

In the cell below write the code to generate an Anime image of a young boy. Make sure to set the seed = `3`.

You can reuse the code in Example 4 but change the prompt to read:

~~~text
# Prompt
prompt = (
    "best_quality, (1boy:1.3), heroic_pose, spiky_hair, intense_eyes, anime_uniform, "
    "dramatic_lighting, cinematic_background, serious_expression, solo, full_body, "
    "anime_style, masterpiece, high_detail, vibrant_colors, wind_effects, glowing_aura, "
    "dynamic_composition, epic_scale, battle_ready, stormy_sky, energy_particles, depth_of_field"
)

~~~

In [ ]:
# Insert your code for Exercise 4 here

import random




If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image17F.png)


### Example 5: Create Anime Scene

The Anime world is full of magical as well as real creatures. One popular Anime TV show that is populated with a variety of creatures is called `Shirokuma Café`.

**Shirokuma Café** (also known as Polar Bear Café) is a Japanese anime and manga series that blends slice-of-life comedy with a whimsical, anthropomorphic twist.
![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image24B.png)

The code in the cell below generates an image that is in `the style of` `Shirokuma Café`. This particular prompt was generated by uploading this image to Microsoft 365 Copilot and asking it to create a prompt to duplicate the image.

In [ ]:
# Example 5: Create Anime Scene

import random

# Set the seed
seed = 8
seed = random.randint(0, 2**32) if seed == -1 else seed
print(f"The seed =", seed)

# Create generator
generator = torch.Generator(device='cuda').manual_seed(int(seed))

# Prompt generated by Microsoft 365 Copilot from Shirokuma Café image
prompt = (
    "best_quality, anime_style, high_detail, vibrant_colors, cute_animal_characters, "
    "panda_sitting_with_pink_swim_ring_and_yellow_shorts, waving_paw, penguin_standing_next_to_panda, "
    "lush_green_foliage_background, stone_wall, summer_theme, friendly_expression, "
    "casual_swimwear, relaxed_atmosphere, slice_of_life_scene, soft_shading"
)

# Negative prompt
neg_prompt = (
    "lowres, bad_anatomy, blurry, distorted_faces, error_limbs, bad_hands, missing_fingers, "
    "extra_limbs, worst_quality, jpeg_artifacts, watermark, cropped, poor_lighting, "
    "oversaturated_colors, unnatural_pose, messy_background, out_of_focus, bad_proportions"
)

# Generate embeddings
prompt_embeds = compel(prompt)
neg_embeds = compel(neg_prompt)

# Generate the image
image = pipe(
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=neg_embeds,
    width=512,
    height=512,
    generator=generator
).images[0]

image

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image18F.png)

Here is the prompt that was used to generate this image:

~~~type
prompt = (
    "best_quality, anime_style, high_detail, vibrant_colors, cute_animal_characters, "
    "panda_sitting_with_pink_swim_ring_and_yellow_shorts, waving_paw, penguin_standing_next_to_panda, "
    "lush_green_foliage_background, stone_wall, summer_theme, friendly_expression, "
    "casual_swimwear, relaxed_atmosphere, slice_of_life_scene, soft_shading"
)
~~~

As you can see the image generated was rather different that one might have expected from the prompt. This kind of _mismatch_ is very common with `text-to-image` generators, even when rather detailed prompts are used.

### **Exercise 5: Create Anime Scene**

In the cell below use exactly the same code that was shown in Example 5.

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image24B.png)

 See if by changing the value of the seed, if you can generate an image that is more similar to the Shirokuma Café image that the prompt was based on.

In [ ]:
# Insert your code for Exercise 5 here



What your output looks like will depend upon the value of your seed.

## **Image-to-Image Pipelines in Stable Diffusion**

#### **Background**

Stable Diffusion is a latent text-to-image diffusion model developed by **Stability AI**, first released in **August 2022**. It quickly became popular due to its open-source nature, high-quality outputs, and ability to run on consumer GPUs.

While the original model focused on **text-to-image generation**, the community and developers soon expanded its capabilities to include **image-to-image** transformations, enabling users to guide generation using existing images.

#### **What Is Image-to-Image in Stable Diffusion?**

Image-to-image (img2img) generation allows users to provide:
- A **source image** (e.g., a sketch, photo, or concept art)
- A **text prompt** describing the desired transformation
- A **strength parameter** controlling how much the output deviates from the input

This technique uses the same latent diffusion process but starts from a **noised version of the input image**, allowing for creative reinterpretation while preserving structure.

### **Evolution of Image-to-Image Pipelines**

##### Stable Diffusion v1.5 (2022)
- First widely used version for img2img tasks.
- Introduced via the `StableDiffusionImg2ImgPipeline` in the `diffusers` library.
- Supported basic transformations with prompt guidance and strength control.

##### Stable Diffusion v2.x (Late 2022–2023)
- Improved resolution and semantic understanding.
- Introduced **depth-to-image** and **inpainting** pipelines.
- Better handling of complex prompts and image conditioning.

##### ControlNet Integration (2023)
- Added fine-grained control using **edge maps**, **pose estimation**, **depth maps**, etc.
- Enabled highly structured transformations while preserving artistic freedom.

##### SDXL (2023–2024)
- Major upgrade with richer visual fidelity and prompt comprehension.
- Image-to-image support extended to **SDXLImg2ImgPipeline**.
- Better performance on high-resolution inputs and nuanced prompts.

### **Parameters in Image-to-Image Pipelines**

| Parameter         | Description                                                                 |
|------------------|-----------------------------------------------------------------------------|
| `prompt`          | Text description guiding the transformation                                 |
| `image`           | Input image to be transformed                                               |
| `strength`        | Controls deviation from input (0.0 = faithful, 1.0 = creative)              |
| `guidance_scale`  | Controls adherence to prompt (higher = more prompt-driven)                  |
| `num_inference_steps` | Number of denoising steps (more = better quality, slower)               |
| `generator`       | Random seed generator for reproducibility                                   |

#### **Use Cases**

- **Artistic reinterpretation** of student sketches
- **Scientific visualization** from concept diagrams
- **Creative storytelling** using visual prompts
- **Biological image transformation** for simulations or hypothetical scenarios


### Setup Image-2-Image Pipeline

The code in the cell below sets up the Image-to-Image pipe line.

In [ ]:
# Setup image-2-image pipeline

from diffusers import StableDiffusionImg2ImgPipeline
from compel import Compel
import torch
from PIL import Image
import requests

# Load the pipeline
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

# Create compel processor
compel = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)

print("Image-to-image pipeline loaded successfully on cuda.")


If the code is correct you should see something like the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image19F.png)

## **Understanding the `strength` Parameter in Image-to-Image Generation**

##### **What Is `strength`?**

In Stable Diffusion's image-to-image pipeline, the `strength` parameter controls **how much the generated image deviates from the input image**. It determines the level of noise added to the input image before the diffusion process begins.

##### **How It Works**

- The image-to-image pipeline works by **encoding the input image into latent space**, adding noise, and then **denoising it guided by the text prompt**.
- The `strength` parameter sets the **starting point** in the denoising process:
  - **Low strength** → less noise → output is **closer to the input image**
  - **High strength** → more noise → output is **more influenced by the prompt**


#### **Typical Values**

| Strength | Behavior                          | Use Case Example                          |
|----------|-----------------------------------|-------------------------------------------|
| 0.1–0.3  | Very close to input image         | Subtle edits, style transfer              |
| 0.4–0.6  | Balanced between input and prompt | Concept transformation                    |
| 0.7–0.9  | Highly creative, prompt-driven    | Abstract reinterpretation, surreal edits  |

> ⚠️ Values above `0.9` may ignore the input image almost entirely.

### Example 6: Image-to-Image Strength Parameter

The code in the cell below uses this rather famous image of an astronaut riding a horse on the moon as the initial image.

![__](https://biologicslab.co/BIO1173/images/class_04/AstronautMoon.jpg)


The **initial image** (also called the **input image**) is the starting point for the image-to-image generation process in Stable Diffusion. It provides the **visual structure** or **content** that the model will transform based on a given **text prompt**.

#### **How It Works**

1. The initial image is **encoded into latent space** using a Variational Autoencoder (VAE).
2. A certain amount of **noise is added**, controlled by the `strength` parameter.
3. The model then **denoises** the image while being guided by the **text prompt**, producing a new image that blends the original content with the prompt's intent.

### **Prompt**

Here is the text prompt for Example 6:

```text
# Prompt
prompt = "a evil clown riding a tiger on the moon"
```

### **Strength Parameter**

In this example we are going to look at how the `strength parameter` affects image creation.

The code in the cell below has the `strength parameter` = `0.01`. This is an extremely low value so we would expect the output image to be relatively unchanged.

In [ ]:
# Example 6: Image-to-Image Strength Parameter

import torch
from IPython.display import display
from PIL import Image
import requests

# Set strength parameter
strength = 0.01  # Creativity level

# Set the seed
seed = 1
print(f"The seed = {seed}")

# Prompt
prompt = "a evil clown riding a tiger on the moon"

# Negative prompt
neg_prompt = "blurry, low quality, distorted"

# Set additional parameters
guidance_scale = 7.5
num_inference_steps = 50

# Validate strength
if not (0.01 <= strength <= 0.9):
    raise ValueError("Strength should be between 0.1 and 0.9 for best results.")

# Create generator
generator = torch.Generator("cuda").manual_seed(seed)

# Load and preprocess initial image
url = "https://biologicslab.co/BIO1173/images/class_04/AstronautMoon.jpg"
init_image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

# Resize and crop to 512x512
init_image = init_image.resize((512, 512))

# Generate embeddings
prompt_embeds = compel(prompt)
neg_embeds = compel(neg_prompt)

# Generate image
try:
    result = pipe(
        prompt_embeds=prompt_embeds,
        negative_prompt_embeds=neg_embeds,
        image=init_image,
        strength=strength,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        generator=generator
    ).images[0]

    # Display result
    display(result)

except Exception as e:
    print("Error during image generation:", e)


If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image29B.png)

As expected the output image is relatively unchanged.

### **Exercise 6A: Image-to-Image Strength Parameter**

In the cell below copy-and-paste the code from Example 6. Keep the `seed` = `1` but set the `strength parameter` = `0.5`. In other words, we want to know what happens when increase the `creativity` to a "medium level"?


In [ ]:
# Insert your code for Exercise 6A here



If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image31B.png)

The horse in the output image seems to be morphing into a more `tiger-like` creature.

### **Exercise 6B: Image-to-Image Strength Parameter**

In the cell below again copy-and-paste Example 6, keep the `seed` = `1` but now set the `strength parameter` = `0.75`.

Let's see how a "high level" of creativity will effect the output image?

In [ ]:
# Insert your code for Exercise 6B here




If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image20F.png)

The output image is now relatively bizzare. The horse has been changed into a tiger albeit a mishapen tiger. The astronaut is also quite different.  

## **Biomedical Use Cases?**

In theory, we could image a wide range of biomedical use cases for the text-to-image Stable Diffusion pipeline demonstrated above. Here are several potential prompts that could, in theory, generate useful images for biomedical education:

```python
# Biomedical prompt
prompt = (
    "highly detailed scientific illustration of a human heart, anatomically accurate, "
    "cross-section view showing chambers and valves, medical textbook style, "
    "labeled diagram aesthetic, clean white background, professional medical illustration, "
    "arteries and veins visible, realistic tissue texture, educational anatomy poster, "
    "high resolution, sharp details, clinical precision"
)

# Cell biology prompt
prompt = (
    "detailed scientific illustration of a eukaryotic cell, cross-section view, "
    "nucleus with chromatin visible, mitochondria, endoplasmic reticulum, golgi apparatus, "
    "ribosomes, cell membrane, medical textbook style, educational diagram, "
    "labeled structure aesthetic, high detail, professional biology illustration"
)

# Neuroscience prompt
prompt = (
    "scientific illustration of a neuron, detailed axon and dendrites, "
    "myelin sheath visible, synaptic terminals, action potential visualization, "
    "medical textbook quality, anatomically accurate, clean background, "
    "neuroscience educational poster, high resolution microscopy style"
)

# Molecular biology prompt
prompt = (
    "3D rendering of DNA double helix structure, accurate base pairs, "
    "phosphate backbone visible, scientific visualization, molecular biology, "
    "high detail protein structure, medical research aesthetic, "
    "clean dark background, glowing nucleotides, educational science poster"
)

# Pathology prompt
prompt = (
    "medical illustration of healthy lung tissue versus diseased tissue comparison, "
    "histology slide aesthetic, microscopic view, alveoli structure visible, "
    "clinical pathology style, educational medical diagram, high detail, "
    "professional medical textbook illustration, accurate cellular structure"
)
```

The ability to rapidly generate high-quality medical illustrations has obvious appeal. In Example 7, we will see if this potential is met?

### Set-up Text-to-Image Pipeline

Run the code in the next cell to setup an text-to-image pipeline to generate biomedical images.

In [ ]:
# Setup text-to-image pipeline for biomedical images

from diffusers import StableDiffusionPipeline
from compel import Compel
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

# Create compel processor
compel = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)

print("Text-to-image pipeline loaded successfully on cuda.")


If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image26F.png)


### Example 7: Biomedical Image Generation

The code in the cell uses the Stable Diffusion text-to-image pipeline `runwayml/stable-diffusion-v1-5` to create a medical illustration showing the difference between health and diseased lung tissue. The prompt used in this example is same `Pathology prompt` shown above.


In [ ]:
# Example 7: Biomedical Image Generation

import random
import torch
from IPython.display import display

# Set the seed
seed = 749991061
seed =random.randint(0, 2**32) if seed == -1 else seed
print(f"The seed = {seed}")

# Create generator
generator = torch.Generator(device='cuda').manual_seed(int(seed))


# Pathology prompt - Motor neuron with action potential initiation
prompt = (
    "medical illustration of healthy lung tissue versus diseased tissue comparison, "
    "histology slide aesthetic, microscopic view, alveoli structure visible, "
    "clinical pathology style, educational medical diagram, high detail, "
    "professional medical textbook illustration, accurate cellular structure"
)

# Negative prompt
neg_prompt = (
    "low quality, blurry, distorted anatomy, incorrect proportions, "
    "abstract, cartoon, anime, watermark, signature, "
    "low resolution, jpeg artifacts, fantasy style, "
    "poorly drawn, amateur, sketch, doodle, multiple neurons, network"
)

# Generate embeddings
prompt_embeds = compel(prompt)
neg_embeds = compel(neg_prompt)

# Generate the image
image = pipe(
    prompt_embeds=prompt_embeds,
    negative_prompt_embeds=neg_embeds,
    width=512,
    height=512,
    num_inference_steps=50,
    guidance_scale=7.5,
    generator=generator
).images[0]

display(image)


If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image24F.png)

Remember, we wanted Stable Diffusion to generate an image showing healthy vs diseased lung tissue. What Stable Diffusion generated is, quite frankky, totally bizzare. Technically, it is a "visual hallucination".

While you can definitely get _really_ different output images from the code above by just changing the value of the `seed`--and you are encouraged to try this--I will bet you a dollar that you will _never_ generate anything that is remotely useful. Believe me, I have tried.

## **Nano Banana Pro for Biomedical Illustrations**

**Nano Banana Pro** was only released by Google around November 20-21, 2025. It is an advanced AI image generation model built on `Gemini 3 Pro Image`. You may recall that we have already used Nano Banana Pro in an earlier lesson.

If you give Nano Banana Pro _exactly_ the same prompt used in Example 7, here is the image that Nano Banana Pro generates:

![__](https://biologicslab.co/BIO1173/images/class_04/class_04_3_image23F.png)

By any objective measure, this is a much more useful image for biomedical education than the image generated by Stable Diffusion.

This raises the following question: "Why is Nano Banana Pro so much better so much better than Stable Diffusion in medical image generation, especially when it comes to labelling the image?"

## **Model Comparison: Nano Banana Pro vs. Stable Diffusion v1.5**

**Nano Banana Pro**, built on the Gemini 3 Pro Image architecture, uses a **multimodal reasoning architecture**. This treats text as semantic symbols to be written. **Stable Diffusion v1.5** utilizes a **Latent Diffusion** architecture that treats text as visual texture to be hallucinated.

#### **1. Architectural Differences**

| Feature | **Nano Banana Pro** (Gemini 3 Pro Image) | **Stable Diffusion v1.5** (RunwayML) |
| :--- | :--- | :--- |
| **Core Mechanism** | **Autoregressive / Reasoning Engine** | **Latent Diffusion (LDM)** |
| **Generation Flow** | **Sequential & Iterative:** Uses a "Plan → Evaluate → Improve" loop. | **Global Denoising:** Refines random noise into an image simultaneously across the entire canvas. |
| **Text Perception** | **Symbolic:** Understands "A-P-P-L-E" as a specific sequence of character tokens. | **Visual:** Sees "text" as a shape pattern (like the texture of brick or leaves). |
| **Resolution** | **Native High-Res (up to 4K):** Renders text directly at target resolution. | **Compressed Latent Space:** Generates at 512x512 (compressed 8x), often crushing fine text details. |


#### **2. Why Nano Banana Pro Generates Correct Text**

Nano Banana Pro does not simply "dream" text based on visual training data. It uses a **token-based validation** system:

*   **Character-by-Character Planning:** It plans the layout of text strings *before* rendering the pixels, ensuring letters appear in the correct order (e.g., "SALE" instead of "SLAE").
*   **No Latent Compression Loss:** It avoids the heavy autoencoder compression (VAE) used by Stable Diffusion, which typically blurs the high-frequency edges required for sharp fonts.
*   **Multimodal Grounding:** It aligns the visual output with internal dictionary knowledge, preventing the "alien hieroglyphics" common in older diffusion models.

#### **3. Why Stable Diffusion v1.5 Fails at Text**

Stable Diffusion v1.5 struggles with text because it lacks a concept of "language":

*   **The "Texture" Problem:** To SD v1.5, a letter is just a shape. It attempts to mimic the *look* of writing without understanding the *spelling*, often resulting in letter-like shapes that form nonsense words.
*   **Spatial Incoherence:** Because it denoises the entire image at once, it struggles to maintain the strict horizontal coherence required for sentences (e.g., letters often merge or float apart).


# **Lesson Turn-In**

When you have completed and run all of the code cells, use the `File --> Print.. --> Microsoft Print to PDF` to generate your PDF if you are running `MS Windows`. If you have a Mac, use the `File --> Print.. --> Save as PDF`

In either case, save your PDF as Copy of Class_04_4.lastname.pdf where lastname is your last name, and upload the file to Canvas.

**NOTE TO WINDOWS USERS:** Your grade will be reduced by 10% if your PDF is found to be missing pages when it is being graded in Canvas. This penalty is simply meant to prevent the grader from having to take the additional steps of (1) downloading your PDF, (2) printing it out using the `Microsoft Print to PDF` and (3) having to resubmit to Canvas so they can grade it

# **Lizard Tail**

### **BACKPROGATION**


![__](https://upload.wikimedia.org/wikipedia/commons/6/60/ArtificialNeuronModel_english.png)


In machine learning, **backpropagation** is a gradient estimation method commonly used for training a neural network to compute its parameter updates.

It is an efficient application of the chain rule to neural networks. Backpropagation computes the gradient of a loss function with respect to the weights of the network for a single input–output example, and does so efficiently, computing the gradient one layer at a time, iterating backward from the last layer to avoid redundant calculations of intermediate terms in the chain rule; this can be derived through dynamic programming.

Strictly speaking, the term backpropagation refers only to an algorithm for efficiently computing the gradient, not how the gradient is used; but the term is often used loosely to refer to the entire learning algorithm – including how the gradient is used, such as by stochastic gradient descent, or as an intermediate step in a more complicated optimizer, such as Adaptive Moment Estimation.[5] The local minimum convergence, exploding gradient, vanishing gradient, and weak control of learning rate are main disadvantages of these optimization algorithms. The Hessian and quasi-Hessian optimizers solve only local minimum convergence problem, and the backpropagation works longer. These problems caused researchers to develop hybrid and fractional optimization algorithms.

Backpropagation had multiple discoveries and partial discoveries, with a tangled history and terminology. See the history section for details. Some other names for the technique include "reverse mode of automatic differentiation" or "reverse accumulation".

## **Intuition**

**Motivation**

The goal of any supervised learning algorithm is to find a function that best maps a set of inputs to their correct output. The motivation for backpropagation is to train a multi-layered neural network such that it can learn the appropriate internal representations to allow it to learn any arbitrary mapping of input to output.

**Learning as an optimization problem**

To understand the mathematical derivation of the backpropagation algorithm, it helps to first develop some intuition about the relationship between the actual output of a neuron and the correct output for a particular training example. Consider a simple neural network with two input units, one output unit and no hidden units, and in which each neuron uses a linear output (unlike most work on neural networks, in which mapping from inputs to outputs is non-linear) that is the weighted sum of its input.

**History**

Backpropagation had been derived repeatedly, as it is essentially an efficient application of the chain rule (first written down by Gottfried Wilhelm Leibniz in 1676) to neural networks.

The terminology "back-propagating error correction" was introduced in 1962 by Frank Rosenblatt, but he did not know how to implement this. In any case, he only studied neurons whose outputs were discrete levels, which only had zero derivatives, making backpropagation impossible.

Precursors to backpropagation appeared in optimal control theory since 1950s. Yann LeCun et al credits 1950s work by Pontryagin and others in optimal control theory, especially the adjoint state method, for being a continuous-time version of backpropagation. Hecht-Nielsen credits the Robbins–Monro algorithm (1951)[23] and Arthur Bryson and Yu-Chi Ho's Applied Optimal Control (1969) as presages of backpropagation. Other precursors were Henry J. Kelley 1960, and Arthur E. Bryson (1961). In 1962, Stuart Dreyfus published a simpler derivation based only on the chain rule. In 1973, he adapted parameters of controllers in proportion to error gradients. Unlike modern backpropagation, these precursors used standard Jacobian matrix calculations from one stage to the previous one, neither addressing direct links across several stages nor potential additional efficiency gains due to network sparsity.

The ADALINE (1960) learning algorithm was gradient descent with a squared error loss for a single layer. The first multilayer perceptron (MLP) with more than one layer trained by stochastic gradient descent was published in 1967 by Shun'ichi Amari.[29] The MLP had 5 layers, with 2 learnable layers, and it learned to classify patterns not linearly separable.

**Modern backpropagation**

Modern backpropagation was first published by Seppo Linnainmaa as "reverse mode of automatic differentiation" (1970) for discrete connected networks of nested differentiable functions.

In 1982, Paul Werbos applied backpropagation to MLPs in the way that has become standard. Werbos described how he developed backpropagation in an interview. In 1971, during his PhD work, he developed backpropagation to mathematicize Freud's "flow of psychic energy". He faced repeated difficulty in publishing the work, only managing in 1981. He also claimed that "the first practical application of back-propagation was for estimating a dynamic model to predict nationalism and social communications in 1974" by him.

Around 1982, David E. Rumelhart independently developed backpropagation and taught the algorithm to others in his research circle. He did not cite previous work as he was unaware of them. He published the algorithm first in a 1985 paper, then in a 1986 Nature paper an experimental analysis of the technique These papers became highly cited, contributed to the popularization of backpropagation, and coincided with the resurging research interest in neural networks during the 1980s.

In 1985, the method was also described by David Parker. Yann LeCun proposed an alternative form of backpropagation for neural networks in his PhD thesis in 1987.

Gradient descent took a considerable amount of time to reach acceptance. Some early objections were: there were no guarantees that gradient descent could reach a global minimum, only local minimum; neurons were "known" by physiologists as making discrete signals (0/1), not continuous ones, and with discrete signals, there is no gradient to take. See the interview with Geoffrey Hinton,[36] who was awarded the 2024 Nobel Prize in Physics for his contributions to the field.

**Early successes**

Contributing to the acceptance were several applications in training neural networks via backpropagation, sometimes achieving popularity outside the research circles.

In 1987, NETtalk learned to convert English text into pronunciation. Sejnowski tried training it with both backpropagation and Boltzmann machine, but found the backpropagation significantly faster, so he used it for the final NETtalk. The NETtalk program became a popular success, appearing on the Today show.

In 1989, Dean A. Pomerleau published ALVINN, a neural network trained to drive autonomously using backpropagation.

The LeNet was published in 1989 to recognize handwritten zip codes.

In 1992, TD-Gammon achieved top human level play in backgammon. It was a reinforcement learning agent with a neural network with two layers, trained by backpropagation.

In 1993, Eric Wan won an international pattern recognition contest through backpropagation.

**After backpropagation**

During the 2000s it fell out of favour, but returned in the 2010s, benefiting from cheap, powerful GPU-based computing systems. This has been especially so in speech recognition, machine vision, natural language processing, and language structure learning research (in which it has been used to explain a variety of phenomena related to first and second language learning.

Error backpropagation has been suggested to explain human brain event-related potential (ERP) components like the N400 and P600.

In 2023, a backpropagation algorithm was implemented on a photonic processor by a team at Stanford University.

## **Backpropagation in Deep Neural Networks**

Backpropagation is a fundamental algorithm used to train deep neural networks. It efficiently computes the **gradient of the loss function** with respect to each weight in the network, enabling the use of **gradient descent** to update the weights and minimize the loss.

![__](https://biologicslab.co/BIO1173/images/class_04/BackProp.jpg)

### Overview of the Process

1. **Forward Pass**: Input data is passed through the network to compute the output (prediction).
2. **Loss Calculation**: The output is compared to the true label using a loss function (e.g., MSE, cross-entropy).
3. **Backward Pass (Backpropagation)**:
   - Gradients of the loss with respect to each parameter are computed using the **chain rule**.
   - These gradients are used to update the weights via an optimization algorithm (e.g., SGD, Adam).

## Mathematical Foundations

Let’s consider a simple feedforward neural network with:
- Input layer: \( x \)
- Hidden layer: \( h = f(Wx + b) \)
- Output layer: \( \hat{y} = g(Vh + c) \)
- Loss function: \( L(\hat{y}, y) \)

### Step 1: Compute Gradients

Using the chain rule:

- Gradient w.r.t. output weights \( V \):
  $$
  \frac{\partial L}{\partial V} = \frac{\partial L}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial V}
  $$

- Gradient w.r.t. hidden weights \( W \):
  $$
  \frac{\partial L}{\partial W} = \frac{\partial L}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial h} \cdot \frac{\partial h}{\partial W}
  $$

### Step 2: Update Weights

Using gradient descent:
$$
V := V - \eta \cdot \frac{\partial L}{\partial V}
$$

$$
W := W - \eta \cdot \frac{\partial L}{\partial W}
$$

Where $\eta$ is the learning rate.


## Backpropagation Algorithm (Pseudocode)

```python
# Assume a simple 2-layer neural network
for epoch in range(num_epochs):
    for x, y in data:
        # Forward pass
        h = f(W @ x + b)
        y_hat = g(V @ h + c)
        loss = compute_loss(y_hat, y)

        # Backward pass
        dL_dyhat = compute_loss_gradient(y_hat, y)
        dL_dV = dL_dyhat @ h.T
        dL_dh = V.T @ dL_dyhat
        dL_dW = (dL_dh * f_prime(W @ x + b)) @ x.T

        # Update weights
        V -= learning_rate * dL_dV
        W -= learning_rate * dL_dW
```

#### **Vectorized Backpropagation**

Below is a minimal, fully vectorized implementation for a 2‑layer perceptron (one hidden layer). It demonstrates the key steps without relying on a deep learning library.


import numpy as np

def tanh(x):
    return np.tanh(x)

def tanh_deriv(x):
    return 1.0 - np.tanh(x)**2

def forward(x, W1, b1, W2, b2):
    z1 = W1 @ x + b1
    a1 = tanh(z1)
    z2 = W2 @ a1 + b2
    a2 = z2          # linear output (e.g., regression)
    cache = (x, z1, a1, z2, a2)
    return a2, cache

def compute_loss(y_true, y_pred):
    return 0.5 * np.sum((y_pred - y_true)**2)

def backward(y_true, cache, W2):
    x, z1, a1, z2, a2 = cache
    
    # Output layer error (MSE + linear)
    delta2 = (a2 - y_true)          # shape (output_dim,)
    
    # Gradients for W2, b2
    grad_W2 = np.outer(delta2, a1)   # shape (output_dim, hidden_dim)
    grad_b2 = delta2                 # shape (output_dim,)
    
    # Hidden layer error
    delta1 = (W2.T @ delta2) * tanh_deriv(z1)   # shape (hidden_dim,)
    
    # Gradients for W1, b1
    grad_W1 = np.outer(delta1, x)              # shape (hidden_dim, input_dim)
    grad_b1 = delta1
    
    grads = {'W1': grad_W1, 'b1': grad_b1,
             'W2': grad_W2, 'b2': grad_b2}
    return grads

#### **Training Loop (SGD)**

```Python
# Dummy data
X = np.random.randn(5, 3)   # 5 samples, 3 features
Y = np.random.randn(5, 1)   # 5 targets

# Initialize parameters
W1 = np.random.randn(4, 3) * 0.01
b1 = np.zeros(4)
W2 = np.random.randn(1, 4) * 0.01
b2 = np.zeros(1)

eta = 0.01   # learning rate

for epoch in range(1000):
    for x, y in zip(X.T, Y.T):   # iterate over samples
        y_hat, cache = forward(x, W1, b1, W2, b2)
        grads = backward(y, cache, W2)
        
        # SGD update
        W1 -= eta * grads['W1']
        b1 -= eta * grads['b1']
        W2 -= eta * grads['W2']
        b2 -= eta * grads['b2']
    
    if epoch % 100 == 0:
        y_pred, _ = forward(X.T, W1, b1, W2, b2)
        loss = compute_loss(Y.T, y_pred)
        print(f'Epoch {epoch}: loss={loss:.4f}')

```

#### **Variants of the Basic Algorithm**


| Variant                | Key Idea                                 | Where It Helps                                 |
|------------------------|------------------------------------------|------------------------------------------------|
| Mini‑Batch SGD         | Update weights after a small batch (32–256 samples) | Faster convergence, better generalization     |
| Momentum               | Keeps running average of gradients       | Accelerates along relevant directions           |
| Adam                   | Adaptive learning rates per parameter    | Works well out‑of‑the‑box for many tasks         |
| Batch Normalization    | Normalizes activations                   | Reduces covariate shift, improves training speed |
| Weight Sharing         | Same weights reused across time/space    | CNNs, RNNs, Transformer attention              |

